In [1]:
print("OK")

OK


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
import os


In [3]:
# Database libraries
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA


In [4]:
api_key = os.getenv('OPENAI_API_KEY')


In [5]:
print(api_key)

sk-2WaDD7SH-SLi8dbvITu97Kt0rnHRpfOKkkH2flns-gT3BlbkFJK_X05dqAGX8q2lkULB9xtOAStiBIM8oFScdnEgrakA


In [6]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data=load_pdf("../data/")

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)

In [38]:
embedding = OpenAIEmbeddings(api_key=api_key)

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [12]:
pip install chromadb==0.5.3


Note: you may need to restart the kernel to use updated packages.


In [13]:
import chromadb
chroma_client = chromadb.Client()

In [14]:
collection = chroma_client.create_collection(name="my_collection")


In [17]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()
vetordb = Chroma.from_documents(documents = text_chunks,
                                embedding=embedding,
                                persist_directory=persist_directory)

In [19]:
vetordb.persist()
vectordb=None

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [20]:
vetordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [21]:
retreiver = vetordb.as_retriever()

In [22]:
docs = retreiver.get_relevant_documents("what are allergies")

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [23]:
len(docs)

4

In [24]:
retreiver = vetordb.as_retriever(search_kwargs={"k":2})

In [26]:
retreiver.search_kwargs

{'k': 2}

In [30]:
qa_chain = RetrievalQA.from_chain_type(llm = OpenAI(),
                                       chain_type="stuff",
                                       retriever=retreiver,
                                       return_source_documents=True)

In [34]:
prompt_template="""

Use the following pieces of information to answer the user's question. If you dont know the anser, just say that you don't know, do not try and piece together knowledge unless you are at least 80% sure that it is the correct answer. If you are less then 80% sure, try and point them in the direction of a good resource to find their answer.
Context: {context}
Question: {question}

Return the helpful answer and nothing else. 
Helpful answer:
"""


In [37]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa_chain({"query": user_input})
    print("Response : ", result["result"])

Response :   Allergies are abnormal reactions of the immune system to otherwise harmless substances. They are among the most common medical disorders, affecting millions of people and leading to school absence and lost productivity in the workplace. They are described as a type of immune reaction in response to allergens, and can be managed through medical treatment or natural remedies.
Response :   I don't have enough information to answer this question.


KeyboardInterrupt: Interrupted by user

In [41]:
vetordb.delete_collection()


ValueError: Collection langchain does not exist.

In [42]:
!rm -rf db/